<center> <h2>A Convolutional Neural Network in Pytorch</h2></center>

In Pytorch, we can use <b>torch.nn</b> package to build a neural network. This package contains the definition and computation of all layers that you need in a neural network.<br>
In which, <b>nn.Module</b> is a base class for all neural networks. Your model should also sub-class of this class and it must contain:
<ul>
    <li>layers,</li>
    <li>a method <b>forward(input)</b> that return the output.</li>
</ul>
Notice that you just have to define the forward function, the backward function is automatically defined for you using autograd.

Let's consider a network to classify the digits:
<img src = "images/example.png" />
The network includes:
<ul>
    <li>2 convolutional layers</li>
    <li>2 max pooling layers</li>
    <li>3 full-connected layers</li>
    <li>All the hidden layers use ReLU as activation function</li>
</ul>

<h2>Let's define the network in Pytorch</h2>

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    '''
        Define the layers in the network
    '''
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    '''
        Implement the forward computation of the network
    '''
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Try to generate an input with the size of 32x32 and use the network to classify.

In [9]:
input = torch.randn(1,1,32,32)
output = net(input)
print(output)

tensor([[ 0.1238, -0.1146, -0.0863, -0.1353, -0.1724,  0.0356, -0.0321,  0.0961,
         -0.0836,  0.0394]], grad_fn=<ThAddmmBackward>)


## The loss functions
There are several different loss functions in <i>nn</i> package. For example, we use <b>nn.MSELoss</b> which compute the mean squared error between the input and the target.

In [12]:
#random a target list and reshape
target = torch.randn(10)
target = target.view(1,-1)

# choose the loss function
loss_function = nn.MSELoss()

# compute the loss between output and target
loss = loss_function(output,target)
print(loss)

tensor(0.7100, grad_fn=<MseLossBackward>)


## Backward propagation and update the weights